<a href="https://colab.research.google.com/github/ashwinvaswani/Generative-Modelling-of-Images-from-Speech/blob/master/src/TIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pydub
!pip install pytube
!pip install mtcnn
!pip install keras-vggface

     |████████████████████████████████| 2.3MB 50kB/s 
  Created wheel for mtcnn: filename=mtcnn-0.0.9-cp36-none-any.whl size=2257690 sha256=84d5411e01203dadb84872db3f1c8780240f690299d9064fa65571a9bd8dc9f0
  Stored in directory: /root/.cache/pip/wheels/85/81/65/6363fa5aafd7a155c896591e0c7c6e27b69642aa82b9cbf076
Successfully built mtcnn


In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2 
import os
import sys
import moviepy.editor as mp
import pickle

import math
from math import ceil
from math import floor

from pydub import AudioSegment
from pytube import YouTube

import shutil

import IPython.display as ipd
import librosa
import librosa.display

import dlib

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from numpy import expand_dims
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2990080/45929032 bytes (6.5%)6897664/45929032 bytes (15.0%)10838016/45929032 bytes (23.6%)14802944/45929032 bytes (32.2%)18694144/45929032 bytes (40.7%)22618112/45929032 bytes (49.2%)26542080/45929032 bytes (57.8%)30474240/45929032 bytes (66.4%)34332672/45929032 bytes (74.8%)38240256/45929032 bytes (83.3%)42016768/45929032 bytes (91.5%)45899776/45929032 bytes (99.9%)

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
PATH = './drive/My Drive/TIP/Dataset/'
PATH_TO_MAIN = './drive/My Drive/TIP/'
YT_LINK = 'www.youtube.com/watch?v='

In [0]:
column_names = ['YouTube_ID', 'start_segment', 'end_segment', 'X_coordinate', 'Y_coordinate']
train_df = pd.read_csv(PATH + 'avspeech_train.csv',names = column_names)
test_df = pd.read_csv(PATH + 'avspeech_test.csv',names = column_names)

In [6]:
train_df.head()

,YouTube_ID,start_segment,end_segment,X_coordinate,Y_coordinate
0,CJoOwXcjhds,233.266000,239.367000,0.780469,0.670833
1,AvWWVOgaMlk,90.000000,93.566667,0.586719,0.311111
2,Y8HMIm8mdns,171.607767,174.607767,0.505729,0.240741
3,akwvpAiLFk0,144.680000,150.000000,0.698438,0.288889
4,Swss72CHSWg,90.023267,97.297200,0.230729,0.204630


In [7]:
test_df.head()

,YouTube_ID,start_segment,end_segment,X_coordinate,Y_coordinate
0,u5MPyrRJPmc,108.2400,111.240000,0.849219,0.305556
1,H1ulMfj5wRY,112.3200,116.940000,0.112500,0.345833
2,-wuxbgMRIWs,30.0300,36.745044,0.744531,0.211111
3,GNRPRH-E-sI,30.2302,38.171467,0.333594,0.494444
4,VvcwAGkSy2o,240.2000,253.366667,0.491667,0.372222


In [8]:
YT_LINK + train_df.iloc[2]["YouTube_ID"]

'www.youtube.com/watch?v=Y8HMIm8mdns'

In [13]:

shutil.rmtree('Images')
shutil.rmtree('Audio')
shutil.rmtree('Videos')
os.mkdir("Images")
os.mkdir("Audio")
os.mkdir("Videos")

len_train = train_df.shape[0]
count_unsuccessful = 0
dict_audio = {}
to_remove = []
for i in range(1,5):

  print(YT_LINK + train_df.iloc[i]["YouTube_ID"])
  if(1):
    yt = YouTube(YT_LINK + train_df.iloc[i]["YouTube_ID"])
    stream = yt.streams.first()
    stream.download(output_path = 'Videos/',filename = train_df.iloc[i]["YouTube_ID"])


    cap = cv2.VideoCapture("Videos/" + train_df.iloc[i]["YouTube_ID"] + '.mp4') 
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    if fps == 0:
      continue

    count = 0

    dir_name = train_df.iloc[i]["YouTube_ID"]
    
    print("FPS is : "  + str(fps))
    start_cnt = train_df.iloc[i]["start_segment"]
    end_cnt = train_df.iloc[i]["end_segment"]
    
    while(1):

        ret,frame = cap.read()

        if ret == 1:

          count += 1

          (h, w) = frame.shape[:2]

          if (((start_cnt + end_cnt)*fps)/2) - 16 < count < (((start_cnt + end_cnt)*fps)/2) + 16:
            dnnFaceDetector = dlib.cnn_face_detection_model_v1( PATH_TO_MAIN + "Face_detection/mmod_human_face_detector.dat")
            faceRects = dnnFaceDetector(frame, 0)
            print(faceRects)

            if len(faceRects) == 0:
              to_remove.append(dir_name)
              continue
            for faceRect in faceRects:
                x1 = faceRect.rect.left()
                y1 = faceRect.rect.top()
                x2 = faceRect.rect.right()
                y2 = faceRect.rect.bottom()
                img = frame[y1:y2, x1:x2]
                print("About to crop")
                print(count)
                img = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
                cv2.imwrite('Images/' +dir_name + '.png', img)
            break


          if count > int(end_cnt*fps):
            break
          k = cv2.waitKey(1)
          if k == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
  # except:
    print("SORRY!!! The video is unavailable")
    count_unsuccessful += 1
    
  try:
    clip = mp.VideoFileClip("Videos/" + train_df.iloc[i]["YouTube_ID"] + '.mp4').subclip(start_cnt,end_cnt)
    clip.audio.write_audiofile("Audio/" + train_df.iloc[i]["YouTube_ID"] +'.mp3') 
    print("before dict")
    dict_audio[train_df.iloc[i]["YouTube_ID"]] = end_cnt - start_cnt
  except:
    print("Error with audio")  

www.youtube.com/watch?v=AvWWVOgaMlk
FPS is : 30.000015705169986
mmod_rectangles[[(532, 80) (871, 420)]]
About to crop
2738
SORRY!!! The video is unavailable
[MoviePy] Writing audio in Audio/AvWWVOgaMlk.mp3


100%|██████████| 79/79 [00:00<00:00, 517.38it/s]

[MoviePy] Done.
before dict


www.youtube.com/watch?v=Y8HMIm8mdns
FPS is : 29.97002997002997
mmod_rectangles[[(574, 114) (710, 251)]]
About to crop
5173
SORRY!!! The video is unavailable
[MoviePy] Writing audio in Audio/Y8HMIm8mdns.mp3


100%|██████████| 67/67 [00:00<00:00, 537.11it/s]

...

100%|██████████| 118/118 [00:00<00:00, 520.34it/s]

[MoviePy] Done.


before dict
www.youtube.com/watch?v=Swss72CHSWg
FPS is : 29.971377246804966
mmod_rectangles[[(252, 108) (331, 187)]]
About to crop
2792
SORRY!!! The video is unavailable
[MoviePy] Writing audio in Audio/Swss72CHSWg.mp3


100%|██████████| 161/161 [00:00<00:00, 824.55it/s]


[MoviePy] Done.
before dict


In [0]:
# Preparing y_train for encoder and x_train for decoder

In [0]:
def get_embeddings(filenames):

  faces = []
  for f in filenames:
    img = cv2.imread(f)
    face_arr = asarray(img)
    faces.append(face_arr)

  samples = asarray(faces,'float32')

  samples = preprocess_input(samples,version=2)

  model = VGGFace(model='resnet50',include_top=False,input_shape=(224,224,3),pooling='avg')

  yhat = model.predict(samples)

  return yhat

In [16]:
face_list = []
for elem in os.listdir('./Images'):
  face_list.append('./Images/' + elem)
y_train_encoder = get_embeddings(face_list)
x_train_decoder = get_embeddings(face_list)














94699520/94694792 [==============================] - 3s 0us/step


In [17]:
y_train_encoder.shape

(4, 2048)

In [0]:
# Using y_train to create decoder

In [0]:
# Input is y_train and output is images from Images folder

In [0]:
# Augmenting Audio

In [18]:
# shutil.rmtree('Processed_Audio')
os.mkdir('Processed_Audio')
temp_dict = dict_audio.copy()
for elem in os.listdir('./Audio'):
  sound = AudioSegment.from_mp3('./Audio/' + elem)
  sound_new = sound
  while temp_dict[elem[:-4]] < 6:
    print("here")
    sound_new += sound
    temp = temp_dict[elem[:-4]]
    temp_dict[elem[:-4]] = 2*temp
    sound = sound_new

  extract = sound[0:6*1000]

  extract.export('Processed_Audio/' + elem, format="mp3")


here
here
here


In [0]:
# to_remove = ['2f32XSMYlDk']
for elem in to_remove:
  os.remove('Processed_Audio/' + elem + '.mp3')

In [0]:
# Preparing x_train

In [21]:
# shutil.rmtree('Spectograms')
os.mkdir('Spectograms')
for filename in os.listdir("./Processed_Audio"):
  audio_path = './Processed_Audio/' + filename
  x , sr = librosa.load(audio_path,sr = 15925)
  print(audio_path)
  print(type(x), type(sr))
  print(x.shape, sr)
  ipd.Audio(audio_path)
  window_size = 25
  window = np.hanning(window_size)
  h1 = 160
  print(h1)

  stft  = librosa.core.spectrum.stft(x, n_fft=512, hop_length=h1,win_length=window_size, window=window)
  out = 2 * np.abs(stft) / np.sum(window_size)
  with open('./Spectograms/' + filename[:-4] + '.pkl','wb') as f:
    pickle.dump(stft,f)
  
  # librosa.display.waveplot(x, sr=sr)
  # mfccs = librosa.feature.mfcc(x, sr=sr)
  # print(mfccs.shape)
  # librosa.display.specshow(mfccs, sr=sr, x_axis='time')
  # print()




./Processed_Audio/akwvpAiLFk0.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/AvWWVOgaMlk.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/Y8HMIm8mdns.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160
./Processed_Audio/Swss72CHSWg.mp3
<class 'numpy.ndarray'> <class 'int'>
(95550,) 15925
160


In [22]:
stft.shape

(257, 598)

In [24]:
for elem in stft:
  print(elem)

[ 2.86977850e-02+0.j  2.55356133e-02+0.j -1.07457330e-02+0.j
  2.23665591e-02+0.j  4.87547778e-02+0.j  9.11378264e-02+0.j
  3.63397188e-02+0.j  7.06664771e-02+0.j  2.53142100e-02+0.j
  1.36666661e-02+0.j  3.19492593e-02+0.j  4.63221334e-02+0.j
  7.37764910e-02+0.j  6.39643148e-02+0.j -3.40951458e-02+0.j
  3.91852446e-02+0.j  3.09302378e-02+0.j  6.96932850e-03+0.j
  6.33110153e-03+0.j  2.08586361e-02+0.j -2.54025590e-02+0.j
 -2.07206625e-02+0.j -2.48295330e-02+0.j  2.08944269e-03+0.j
  4.57713939e-02+0.j -1.03576347e-01+0.j -4.01646867e-02+0.j
 -5.73446825e-02+0.j -1.99216660e-02+0.j -2.33364645e-02+0.j
 -7.55458996e-02+0.j -2.61680502e-02+0.j -6.00997359e-03+0.j
 -1.73075140e-01+0.j  1.64365679e-01+0.j -3.71296495e-01+0.j
 -2.64995873e-01+0.j  5.94325960e-01+0.j  5.93625903e-01+0.j
 -5.02649605e-01+0.j -3.71454477e-01+0.j -3.77077788e-01+0.j
 -3.23134363e-01+0.j  3.12362541e-03+0.j  1.81688368e-01+0.j
  1.96270928e-01+0.j  2.25754172e-01+0.j -4.44436446e-02+0.j
 -1.53114766e-01+0.j -6.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
# Todo: For all pickle files in spectogram 

In [0]:
def get_encoder_network_input(k):
  shape_x = k.shape[0]
  shape_y = k.shape[1]
  mod = np.zeros((shape_x,shape_y))
  for i in range(shape_x):
    for j in range(shape_y):
      mod[i][j] = np.abs(k[i][j])

  theta_ = np.zeros((shape_x,shape_y))
  for i in range(shape_x):
    for j in range(shape_y):
      theta_[i][j] = np.angle(k[i][j])

  real = np.zeros((shape_x,shape_y))
  complex_ = np.zeros((shape_x,shape_y))
  for i in range(shape_x):
    for j in range(shape_y):
      temp = mod[i][j]
      real[i][j] = temp * math.cos(complex_[i][j])
      complex_[i][j] = temp * math.sin(complex_[i][j])

  real = np.expand_dims(real,axis = 0)
  complex_ = np.expand_dims(complex_,axis = 0)
  combined = np.concatenate((real,complex_))

  return combined

In [16]:
encoder_input = []
for elem in os.listdir('./Spectograms/'):
  with open('./Spectograms/' + elem,'rb') as f:
    print('./Spectograms/' + elem)
    k = pickle.load(f)
    print(k.shape)

  encoder_input.append(np.transpose(get_encoder_network_input(k)))

./Spectograms/labiHToR5nk.pkl
(257, 301)
./Spectograms/zFH0QbS-l-w.pkl
(257, 301)
./Spectograms/h5wT_c4fQ1o.pkl
(257, 301)
./Spectograms/IrXrbrZWflA.pkl
(257, 301)
./Spectograms/lcClO5lHEjA.pkl
(257, 301)
./Spectograms/uCGKDCWxqxo.pkl
(257, 301)
./Spectograms/rCp8Jae81KU.pkl
(257, 301)
./Spectograms/JjduaMIoKvI.pkl
(257, 301)
./Spectograms/7rYeSDHS0U0.pkl
(257, 301)
./Spectograms/qzM4wshoqGs.pkl
(257, 301)
./Spectograms/aaEA__Js2u0.pkl
(257, 301)
./Spectograms/8nQBG5hvjpk.pkl
(257, 301)
./Spectograms/xbgfxIc-nbs.pkl
(257, 301)
./Spectograms/sujFCXbYkMo.pkl
(257, 301)
./Spectograms/YXcVkIEMGds.pkl
(257, 301)
./Spectograms/AvWWVOgaMlk.pkl
(257, 301)
./Spectograms/tdTXVU5wN8I.pkl
(257, 301)
./Spectograms/UBL2Vowiulk.pkl
(257, 301)
./Spectograms/BjvtZkHWExY.pkl
(257, 301)
./Spectograms/Cy9SUMj5wGY.pkl
(257, 301)
./Spectograms/R0u9E8GsUXk.pkl
(257, 301)
./Spectograms/PmD-LzPS2rg.pkl
(257, 301)
./Spectograms/QoQF8N5ZsQA.pkl
(257, 301)
./Spectograms/3AsPqH3QaQM.pkl
(257, 301)
./Spectograms/8E

In [17]:
len(encoder_input)

42

In [0]:
# For encoder

# Input : encoder input as np array
# Output : y_train_encoder

# FOr decoder

# Input : x_train_encoder
# Output : Images from Images folder

In [0]:
x_enc_train = np.asarray(encoder_input)

In [20]:
x_enc_train.shape

(42, 301, 257, 2)

In [25]:
y_train_encoder.shape

(42, 2048)

In [0]:
with open(PATH_TO_MAIN + 'pickles/encoder_trainX.pkl','wb') as f:
    pickle.dump(x_enc_train,f)

In [0]:
with open(PATH_TO_MAIN + 'pickles/encoder_trainY.pkl','wb') as f:
    pickle.dump(y_train_encoder,f)

In [0]:
from google.colab import files
files.download('encoder_trainY.pkl')

In [1]:
!nvidia-smi

Thu Nov  7 19:05:12 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  